In [17]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sortedcontainers import SortedList
from sklearn.model_selection import train_test_split
import math
from datetime import datetime
import re



In [2]:
df = pd.read_csv('../dataset/test.tours.csv')
df = shuffle(df)

In [3]:
for i in range(len(df)):
        period = df['period'][i]
        try:
            days = int(period.split('ngày')[0])
        except:
            days = 0
        try:
            nights = int(period.split('đêm')[0])
        except:
            nights = 0
        total_hours = (days + nights) * 24
        df.at[i, 'period'] = total_hours

In [7]:
df.head(1)

,_id,name,description,price,departureLocation,period,images,destination,departureDay,createdAt,__v,numLikes
294,65db0c2d0b1aa37831a6b6a3,Du lịch Cao Cấp Quy Nhơn - Bình Định - KDL Hầm...,Hành trình: Quy Nhơn - Bình Định - KDL Hầm Hồ ...,8019296,Hồ Chí Minh,72,//dulichviet.com.vn/images/bandidau/NOI-DIA/Qu...,Du lịch Tết 2022 - Bảo Tàng Quang Trung - KDL ...,2024-03-04T00:00:00.000Z,2024-02-25T09:45:17.971Z,0.0,1.0


In [8]:
tours = {}
for row in df.values:
  _id, name,description, price, departureLocation, period, images,destination,departureDay,createdAt,__v,numLikes = row
  tours[_id] = {
      'id': _id,
      'name': name,
      'description': description,
      'price': price,
      'departureLocation': departureLocation,
      'period': period,
      'images': images,
      'destination':destination,
      'departureDay': departureDay,
      'createdAt':createdAt,
      '__v': __v,
      'numLikes':numLikes
  }

In [20]:
def get_sim_name(tour_id_i,tour_id_j):
    name_i = tours[tour_id_i]['name']
    name_j = tours[tour_id_j]['name']

    tokens_i = re.findall(r'\w+', name_i.lower())  
    tokens_j = re.findall(r'\w+', name_j.lower())
    common_tokens = set(tokens_i) & set(tokens_j)
    similarity = len(common_tokens) / len(name_i)

    return similarity * 0.2

def get_sim_departureLocation(tour_id_i,tour_id_j):
    departureLocation_i = set(tours[tour_id_i]['departureLocation'])
    departureLocation_j = set(tours[tour_id_j]['departureLocation'])
    similarity = len(departureLocation_i.intersection(departureLocation_j))/len(departureLocation_i.union(departureLocation_j))
    return  similarity * 0.2

def get_sim_price(tour_id_i,tour_id_j):
    price_i = tours[tour_id_i]['price']
    price_j = tours[tour_id_j]['price']
    if price_i == 0 or price_j == 0:
            return 0
    max_price = max(price_i, price_j)

    normalized_price_i = price_i / max_price
    normalized_price_j = price_j / max_price

    normalized_price_i = (max_price- price_i ) / (max_price )
    normalized_price_j = (max_price - price_j) / (max_price )

    similarity = abs(normalized_price_i - normalized_price_j)

    return similarity * 0.2

def get_sim_period(tour_id_i,tour_id_j):
    period_i = tours[tour_id_i]['period']
    period_j = tours[tour_id_j]['period']
    diff = abs(period_i - period_j)

    if diff < 12:
        return 0.2
    elif diff < 24:
        return 0.2 * 0.8
    elif diff < 48:
        return 0.2 * 0.6
    return 0

def get_sim_departureDay(tour_id_i,tour_id_j):
    departure_day_i = datetime.fromisoformat(tours[tour_id_i]['departureDay'].split('T')[0])
    departure_day_j = datetime.fromisoformat(tours[tour_id_j]['departureDay'].split('T')[0])

    similarity = 1 / (1 + abs((departure_day_i - departure_day_j).days))

    return similarity * 0.2

def get_tour_similarities(tour_id_i,tour_id_j):
    sim_name = get_sim_name(tour_id_i,tour_id_j)
    sim_departureLocation = get_sim_departureLocation(tour_id_i,tour_id_j)
    sim_price = get_sim_price(tour_id_i,tour_id_j)
    sim_period = get_sim_period(tour_id_i,tour_id_j)
    sim_departureDay = get_sim_departureDay(tour_id_i,tour_id_j)

    print('sim_name',sim_name)
    print('sim_departureLocation',sim_departureLocation)
    print('sim_price',sim_price)
    print('sim_period',sim_period)
    print('sim_departureDay',sim_departureDay)
    return sim_name + sim_departureLocation + sim_price + sim_period + sim_departureDay

In [21]:
# get tour similarities
tour_id = '65cc76cf4da78f7cdfcc4f5b'
k = 10
list = SortedList()
for id in tours.keys():
  if(id == tour_id):
    continue
  sim = get_tour_similarities(tour_id,id)
  list.add((sim, tours[id]['name']))
  if len(list) > k:
    del list[0]
sorted_list = sorted(list, reverse=True)  # Sắp xếp danh sách theo độ tương tự giảm dần
top_k = sorted_list[:k]  # Lấy ra k phần tử đầu tiên
top_k

sim_name 0.006666666666666667
sim_departureLocation 0.05
sim_price 0.0977130612244898
sim_period 0.12
sim_departureDay 0.010526315789473684
sim_name 0.015555555555555557
sim_departureLocation 0.05
sim_price 0.1221938775510204
sim_period 0.2
sim_departureDay 0.011764705882352941
sim_name 0.013333333333333334
sim_departureLocation 0.05
sim_price 0.0977130612244898
sim_period 0.12
sim_departureDay 0.006666666666666667
sim_name 0.008888888888888889
sim_departureLocation 0.05
sim_price 0.0977130612244898
sim_period 0
sim_departureDay 0.011111111111111112
sim_name 0.006666666666666667
sim_departureLocation 0.05
sim_price 0.0977130612244898
sim_period 0.2
sim_departureDay 0.011111111111111112
sim_name 0.008888888888888889
sim_departureLocation 0.05
sim_price 0.0977130612244898
sim_period 0.12
sim_departureDay 0.006896551724137931
sim_name 0.008888888888888889
sim_departureLocation 0.05
sim_price 0.0977130612244898
sim_period 0.2
sim_departureDay 0.005263157894736842
sim_name 0.008888888888888

[(0.5718157262905161,
  'Du lịch Hạ Long Tết Âm lịch - Yên Tử - Sapa - Land Tour từ Hà Nội 2024'),
 (0.5394135880424777,
  'Du lịch Đa Nẵng - Huế - Động Thiên Đường mùa Xuân 4 ngày từ Hà Nội 2024'),
 (0.5363052721088436,
  'Du lịch Đông Bắc mùa Xuân - Hà Nội - Hà Giang - Quản Bạ - Sông Nho Quế 4 ngày từ Hà Nội 2024'),
 (0.5363052721088436,
  'Du lịch Đông Bắc mùa Xuân - Hà Nội - Hà Giang - Quản Bạ - Sông Nho Quế 4 ngày từ Hà Nội 2024'),
 (0.5330360370220696,
  'Du lịch Đa Nẵng - Huế - Động Thiên Đường mùa Xuân 4 ngày từ Hà Nội 2024'),
 (0.53241812318987,
  'Du lịch Đà Nẵng - Huế - Làng Hương Thuỷ Xuân - Mikazuki mùa Xuân từ Hà Nội 2024'),
 (0.5175170068027211,
  'Du lịch Tết Âm lịch - Tour Thái Lan - Thiên Đường Biển Phuket - Đảo Phi Phi từ Sài Gòn 2024'),
 (0.47830312388113144,
  'Du lịch Lào - Viêng Chăn - Luang Prabang - Vang Viêng mùa Xuân từ Hà Nội 2024'),
 (0.4610010430839002,
  'Du lịch Đông Bắc Hà Nội - Hà Giang - Cao Bằng - Thác Bản Giốc dịp Lễ 2/9 từ Hà Nội 2023'),
 (0.460131